# This is running the K-Nearest Neighbour (KNN) models, all dataset folders at once.  

The model takes the image data from a root folder (greyscale/gy. original(RBG)/og or Four-channel/fc and runs them through a random forest model.

VERSIONS

Outputs are to the 3 (_gy, _or, _fc) data folders which themselves reflect the quality of the data provided.

PARAMETERS

It adjusts the data to the 3 datasizes used in the KNN and Random Forest Models (64x64,128x128,256x256)

The 3x2 grid search is number of neighhours x weights

The model uses 5-fold on the 80% 'training-validation' data.  The data includes 20% for final testing.

LOCALISATION

root_folders - these are the sources of data (right at the end of each code block)
Adjust as needed.

PIP installs are provided below

TIMING

The models (when run in series and with concurrent cores) ran in 2 hr 23 mins.
This is indicative as depends on the resources available.

RESULTS

each model x imagesize combination generates an excel file with results and logs
These are saved in the related root folders.


In [1]:
# ###Installations could be required locally to run this and later code.  Unhash the below to install ####

# pip install pandas xlsxwriter numpy opencv-python scikit-learn matplotlib seaborn



   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   --------- ----------------------------- 41.0/159.9 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 159.9/159.9 kB 2.4 MB/s eta 0:00:00


In [1]:
import os
import numpy as np
import cv2
import time
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from datetime import datetime
import concurrent.futures
import sys

# Function to load and preprocess images
def load_images_from_folder(folder, target_size):
    images = []
    labels = []
    class_names = os.listdir(folder)
    filenames = []
    for label, class_name in enumerate(class_names):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            if img is not None:
                # Handle different bit depths
                if len(img.shape) == 2:  # Grayscale image
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                elif img.shape[2] == 4:  # 4-channel image (e.g., RGBA)
                    img = img[:, :, :3]  # Discard the alpha channel

                img = cv2.resize(img, target_size)  # Resize to target_size
                images.append(img)
                labels.append(label)
                filenames.append(filename)
    return np.array(images), np.array(labels), class_names, filenames

# Function to preprocess and flatten images
def preprocess_images(X, target_size):
    X_resized = []
    for img in X:
        img_resized = cv2.resize(img, target_size)
        X_resized.append(img_resized)
    X_resized = np.array(X_resized).astype('float32') / 255.0
    return X_resized.reshape((X_resized.shape[0], -1))

# Define parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 3, 5],
    'weights': ['uniform', 'distance'],
    'metric': ['hamming'],
    'algorithm': ['auto']
}

image_sizes = [(64, 64), (128, 128), (256, 256)]

def run_model(train_dataset_folder, test_dataset_folder, model_type, root_folder_suffix):
    # Record start time for the model processing
    model_start_time = datetime.now()
    log = []
    log.append(f"Processing {root_folder_suffix}:{model_type} - Started at {model_start_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Started at {model_start_time}")

    # Load and preprocess training data
    log.append(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing training data")
    print(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing training data")
    start_time = time.time()
    X_train_orig, y_train, class_names, train_filenames = load_images_from_folder(train_dataset_folder, target_size=(128, 128))
    load_preprocess_time = time.time() - start_time
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for loading and preprocessing: {load_preprocess_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for loading and preprocessing: {load_preprocess_time:.2f} seconds")

    best_params = None
    best_score = 0
    best_target_size = None
    best_conf_matrix = None
    best_X_train_resized = None

    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    timing_results = []
    cm_results = []

    for target_size in image_sizes:
        start_size_time = time.time()
        log.append(f"Processing {root_folder_suffix}:{model_type} - Evaluating for image size: {target_size}")
        print(f"Processing {root_folder_suffix}:{model_type} - Evaluating for image size: {target_size}")

        X_train_resized = preprocess_images(X_train_orig, target_size)

        for n_neighbors in param_grid['n_neighbors']:
            for weights in param_grid['weights']:
                for metric in param_grid['metric']:
                    for algorithm in param_grid['algorithm']:
                        params = {
                            'n_neighbors': n_neighbors,
                            'weights': weights,
                            'metric': metric,
                            'algorithm': algorithm
                        }
                        knn = KNeighborsClassifier(**params)

                        start_grid_item_time = time.time()
                        scores = cross_val_score(knn, X_train_resized, y_train, cv=kf, scoring='accuracy')
                        grid_item_time = time.time() - start_grid_item_time

                        mean_score = scores.mean()
                        timing_results.append({
                            'image_size': target_size,
                            'n_neighbors': n_neighbors,
                            'weights': weights,
                            'metric': metric,
                            'algorithm': algorithm,
                            'mean_score': mean_score,
                            'time': grid_item_time,
                            'scoring_metric': 'accuracy'
                        })

                        log.append(f"Processing {root_folder_suffix}:{model_type} - Params: {params} - Mean Score (accuracy): {mean_score:.4f} - Time: {grid_item_time:.2f} seconds")
                        print(f"Processing {root_folder_suffix}:{model_type} - Params: {params} - Mean Score (accuracy): {mean_score:.4f} - Time: {grid_item_time:.2f} seconds")

                        # Collect confusion matrix for the current model
                        X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train_resized, y_train, test_size=0.2, random_state=42)
                        knn.fit(X_train_split, y_train_split)
                        y_valid_pred = knn.predict(X_valid_split)
                        cm = confusion_matrix(y_valid_split, y_valid_pred)
                        cm_results.append({
                            'image_size': target_size,
                            'params': params,
                            'confusion_matrix': cm
                        })

                        if mean_score > best_score:
                            best_score = mean_score
                            best_params = params
                            best_target_size = target_size
                            best_conf_matrix = cm
                            best_X_train_resized = X_train_resized

        size_time = time.time() - start_size_time
        log.append(f"Processing {root_folder_suffix}:{model_type} - Total time taken for image size {target_size}: {size_time:.2f} seconds\n")
        print(f"Processing {root_folder_suffix}:{model_type} - Total time taken for image size {target_size}: {size_time:.2f} seconds\n")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Score: {best_score}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Parameters: {best_params}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Image Size: {best_target_size}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Score: {best_score}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Parameters: {best_params}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Image Size: {best_target_size}")

    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    result_folder = os.path.dirname(train_dataset_folder)
    file_path = os.path.join(result_folder, f"{root_folder_suffix}_{model_type}_KNN_results_{timestamp}.xlsx")

    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        # Save best parameters
        best_params_df = pd.DataFrame([best_params])
        best_params_df['image_size'] = [best_target_size]
        best_params_df.to_excel(writer, sheet_name='Best_Parameters', index=False)

        # Save timing results
        timing_df = pd.DataFrame(timing_results)
        timing_df.to_excel(writer, sheet_name='Timing_Results', index=False)

        # Save confusion matrices for all grid search iterations
        cm_data = []
        for cm_result in cm_results:
            for i in range(len(class_names)):
                for j in range(len(class_names)):
                    cm_data.append({
                        'image_size': cm_result['image_size'],
                        'params': cm_result['params'],
                        'true_class': class_names[i],
                        'predicted_class': class_names[j],
                        'count': cm_result['confusion_matrix'][i, j]
                    })
        cm_df = pd.DataFrame(cm_data)
        cm_df.to_excel(writer, sheet_name='Confusion_Matrices', index=False)

    log.append(f"Processing {root_folder_suffix}:{model_type} - Grid search results saved to {file_path}")
    print(f"Processing {root_folder_suffix}:{model_type} - Grid search results saved to {file_path}")

    # Load the test dataset with best image size
    log.append(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing test data")
    print(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing test data")
    X_test_orig, y_test, _, test_filenames = load_images_from_folder(test_dataset_folder, target_size=best_target_size)

    # Preprocess test images
    X_test_resized = preprocess_images(X_test_orig, best_target_size)

    # Evaluate the final model on the test dataset
    start_time = time.time()
    final_model = KNeighborsClassifier(**best_params)
    final_model.fit(best_X_train_resized, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_test_pred = final_model.predict(X_test_resized)
    predict_time = time.time() - start_time

    test_accuracy = accuracy_score(y_test, y_test_pred)
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Accuracy: {test_accuracy:.4f}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for final training: {train_time:.2f} seconds")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for final prediction: {predict_time:.2f} seconds")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Classification Report:\n{classification_report(y_test, y_test_pred, target_names=class_names)}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Confusion Matrix:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Accuracy: {test_accuracy:.4f}")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for final training: {train_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for final prediction: {predict_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Classification Report:\n{classification_report(y_test, y_test_pred, target_names=class_names)}")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Confusion Matrix:\n{confusion_matrix(y_test, y_test_pred)}")

    # Save final test results
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        # Save test classification report
        test_report = classification_report(y_test, y_test_pred, target_names=class_names, output_dict=True)
        test_report_df = pd.DataFrame(test_report).transpose()
        test_report_df.to_excel(writer, sheet_name='Test_Classification_Report')

        # Save test confusion matrix
        test_cm = confusion_matrix(y_test, y_test_pred)
        test_cm_df = pd.DataFrame(test_cm, index=class_names, columns=class_names)
        test_cm_df.to_excel(writer, sheet_name='Test_Confusion_Matrix')

        # Save image predictions
        predictions_df = pd.DataFrame({
            'Filename': test_filenames,
            'Actual': [class_names[label] for label in y_test],
            'Predicted': [class_names[label] for label in y_test_pred]
        })
        predictions_df.to_excel(writer, sheet_name='Image_Predictions', index=False)

        # Save summary
        summary_data = {
            'Best Score': [best_score],
            'Best Parameters': [best_params],
            'Best Image Size': [best_target_size],
            'Test Accuracy': [test_accuracy],
            'Training Time': [train_time],
            'Prediction Time': [predict_time]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)

        # Save log
        log_df = pd.DataFrame(log, columns=['Log'])
        log_df.to_excel(writer, sheet_name='Log', index=False)

    # Record end time for the model processing
    model_end_time = datetime.now()
    log.append(f"Processing {root_folder_suffix}:{model_type} - Ended at {model_end_time}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Total duration: {model_end_time - model_start_time}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Parameters identified from the grid search: {best_params}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Image Size identified from the grid search: {best_target_size}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Final test results saved to {file_path}")
    print(f"Processing {root_folder_suffix}:{model_type} - Ended at {model_end_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Total duration: {model_end_time - model_start_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Parameters identified from the grid search: {best_params}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Image Size identified from the grid search: {best_target_size}")
    print(f"Processing {root_folder_suffix}:{model_type} - Final test results saved to {file_path}")

# Function to iterate over the data directories and run the model
def run_models_for_all_datasets(root_folders):
    # Record start time for the overall processing
    overall_start_time = datetime.now()
    print(f"Overall processing started at {overall_start_time}")

    model_types = ['country', 'exact_piece', 'force', 'piece']

    tasks = []

    for root_folder in root_folders:
        root_folder_suffix = os.path.basename(root_folder)[-2:]
        for model_type in model_types:
            train_folder = os.path.join(root_folder, f"{model_type}_train")
            test_folder = os.path.join(root_folder, f"{model_type}_test")

            if os.path.exists(train_folder) and os.path.exists(test_folder):
                tasks.append((train_folder, test_folder, model_type, root_folder_suffix))
            else:
                print(f"Skipping {model_type} in {root_folder} as train or test folder does not exist")

    # Run models in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(run_model, train, test, model, suffix) for train, test, model, suffix in tasks]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as exc:
                print(f"Generated an exception: {exc}")

    # Record end time for the overall processing
    overall_end_time = datetime.now()
    print(f"Overall processing ended at {overall_end_time}")
    print(f"Total duration for all models: {overall_end_time - overall_start_time}")

# List of root folders
root_folders = [
    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\06.DataSets_gy',
    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\07.DataSets_or',
    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\08.DataSets_fc'
]

# Run the models for all datasets
run_models_for_all_datasets(root_folders)


Overall processing started at 2024-08-05 17:36:27.611633
Processing gy:country - Started at 2024-08-05 17:36:27.612837
Processing gy:country - Loading and preprocessing training data
Processing gy:exact_piece - Started at 2024-08-05 17:36:27.612837
Processing gy:exact_piece - Loading and preprocessing training data
Processing gy:force - Started at 2024-08-05 17:36:27.612837
Processing gy:force - Loading and preprocessing training data
Processing gy:piece - Started at 2024-08-05 17:36:27.613859
Processing gy:piece - Loading and preprocessing training data
Processing or:country - Started at 2024-08-05 17:36:27.613859
Processing or:country - Loading and preprocessing training data
Processing or:exact_piece - Started at 2024-08-05 17:36:27.613859
Processing or:exact_piece - Loading and preprocessing training data
Processing or:force - Started at 2024-08-05 17:36:27.615132
Processing or:force - Loading and preprocessing training data
Processing or:piece - Started at 2024-08-05 17:36:27.6171

Processing fc:exact_piece - Params: {'n_neighbors': 1, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7369 - Time: 31.74 seconds
Processing fc:piece - Params: {'n_neighbors': 1, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.8807 - Time: 33.17 seconds
Processing or:piece - Params: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.8586 - Time: 30.44 seconds
Processing or:country - Params: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7247 - Time: 32.55 seconds
Processing or:exact_piece - Params: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7051 - Time: 33.79 seconds
Processing or:force - Params: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.9600 - Ti

Processing fc:exact_piece - Total time taken for image size (64, 64): 227.00 seconds

Processing fc:exact_piece - Evaluating for image size: (128, 128)
Processing fc:country - Total time taken for image size (64, 64): 228.81 seconds

Processing fc:country - Evaluating for image size: (128, 128)
Processing fc:force - Total time taken for image size (64, 64): 228.89 seconds

Processing fc:force - Evaluating for image size: (128, 128)
Processing fc:piece - Total time taken for image size (64, 64): 230.98 seconds

Processing fc:piece - Evaluating for image size: (128, 128)
Processing gy:country - Params: {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7761 - Time: 112.78 seconds
Processing gy:exact_piece - Params: {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7459 - Time: 119.44 seconds
Processing gy:force - Params: {'n_neighbors': 1, 'weights': 'uniform', 'metric':

Processing or:force - Params: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.9755 - Time: 133.45 seconds
Processing fc:exact_piece - Params: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7426 - Time: 132.32 seconds
Processing fc:country - Params: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7622 - Time: 131.94 seconds
Processing fc:piece - Params: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.8905 - Time: 132.45 seconds
Processing fc:force - Params: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.9755 - Time: 130.60 seconds
Processing or:exact_piece - Params: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.

Processing fc:exact_piece - Params: {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7435 - Time: 1151.34 seconds
Processing gy:country - Params: {'n_neighbors': 1, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7712 - Time: 928.36 seconds
Processing gy:force - Params: {'n_neighbors': 1, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.9730 - Time: 906.85 seconds
Processing gy:exact_piece - Params: {'n_neighbors': 1, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7361 - Time: 970.84 seconds
Processing gy:piece - Params: {'n_neighbors': 1, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.8848 - Time: 1008.58 seconds
Processing or:piece - Params: {'n_neighbors': 1, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0

Processing or:force - Params: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.9575 - Time: 1086.53 seconds
Processing gy:country - Params: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.6944 - Time: 967.38 seconds
Processing gy:piece - Params: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.8611 - Time: 1021.06 seconds
Processing gy:force - Params: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.9518 - Time: 978.34 seconds
Processing gy:exact_piece - Params: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.6658 - Time: 1008.26 seconds
Processing gy:country - Total time taken for image size (256, 256): 6897.46 seconds

Processing gy:country - Best Score: 0.7761425225828036
Proce

Processing gy:exact_piece - Ended at 2024-08-05 19:52:17.511320
Processing gy:exact_piece - Total duration: 2:15:49.898483
Processing gy:exact_piece - Best Parameters identified from the grid search: {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'}
Processing gy:exact_piece - Best Image Size identified from the grid search: (128, 128)
Processing gy:exact_piece - Final test results saved to C:\Users\ReCas\OneDrive\Documents\2024_AIMachineLearning\99_Projects\06.DataSets_gy\gy_exact_piece_KNN_results_20240805_195121.xlsx
Processing fc:country - Params: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.7116 - Time: 821.76 seconds
Processing or:piece - Params: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'hamming', 'algorithm': 'auto'} - Mean Score (accuracy): 0.8750 - Time: 832.34 seconds
Processing fc:piece - Params: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'hamming', 'algorit

Processing fc:piece - Ended at 2024-08-05 19:58:53.323603
Processing fc:piece - Total duration: 2:22:25.700459
Processing fc:piece - Best Parameters identified from the grid search: {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'hamming', 'algorithm': 'auto'}
Processing fc:piece - Best Image Size identified from the grid search: (128, 128)
Processing fc:piece - Final test results saved to C:\Users\ReCas\OneDrive\Documents\2024_AIMachineLearning\99_Projects\08.DataSets_fc\fc_piece_KNN_results_20240805_195804.xlsx
Processing fc:force - Test Accuracy: 0.9837
Processing fc:force - Time taken for final training: 0.40 seconds
Processing fc:force - Time taken for final prediction: 23.47 seconds
Processing fc:force - Test Classification Report:
              precision    recall  f1-score   support

         AIR       0.96      0.97      0.96        68
         LND       0.99      0.97      0.98        68
         SEA       0.99      0.99      0.99       170

    accuracy                  

Processing fc:exact_piece - Test Accuracy: 0.8007
Processing fc:exact_piece - Time taken for final training: 0.37 seconds
Processing fc:exact_piece - Time taken for final prediction: 18.27 seconds
Processing fc:exact_piece - Test Classification Report:
                  precision    recall  f1-score   support

101_100RUS_01INF       0.67      0.86      0.75         7
102_100RUS_02TNK       1.00      0.57      0.73         7
103_100RUS_03FGT       0.86      0.86      0.86         7
104_100RUS_04BMB       0.78      1.00      0.88         7
105_100RUS_05DES       0.57      0.67      0.62         6
106_100RUS_06TRS       1.00      0.71      0.83         7
107_100RUS_07SUB       0.86      0.86      0.86         7
108_100RUS_08BAT       1.00      1.00      1.00         7
109_100RUS_09CAR       0.75      1.00      0.86         6
201_200GER_01INF       0.86      0.86      0.86         7
202_200GER_02TNK       0.83      0.71      0.77         7
203_200GER_03FGT       0.75      0.86      0.80   